In [48]:
# Zavier Morales
# Edouard Mason
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler

RANDOM_STATE_ID = 100577770

# TODO: divide all steps into separate blocks
#   Also write out each ML type manually, don't loop thru them

# Model 1: KNN

# EDA
# load dataset


raw_data = pd.read_pickle('/Users/edouardmason/Downloads/bank_06.pkl')
# find number of instances
print(raw_data.shape)
print(raw_data.head())
# if there is missing data

# regression or classification -> 
# which vars are categoric/numeric

# find cardinality
scaler = MinMaxScaler()

pipe = Pipeline([
    ('scalar', scaler),
    ('')
])

#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE_ID)

(11000, 17)
   age         job  marital  education default  balance housing loan  contact  \
0   59      admin.  married  secondary      no     2343     yes   no  unknown   
1   56      admin.  married  secondary      no       45      no   no  unknown   
2   41  technician  married  secondary      no     1270     yes   no  unknown   
3   55    services  married  secondary      no     2476     yes   no  unknown   
4   54      admin.  married   tertiary      no      184      no   no  unknown   

   day month  duration  campaign  pdays  previous poutcome deposit  
0    5   may      1042         1     -1         0  unknown     yes  
1    5   may      1467         1     -1         0  unknown     yes  
2    5   may      1389         1     -1         0  unknown     yes  
3    5   may       579         1     -1         0  unknown     yes  
4    5   may       673         2     -1         0  unknown     yes  


## **EDA**

In [49]:
print("Shape of the dataset:\n", raw_data.shape)
print("\nFirst few rows:\n",raw_data.head())
print("\nList of columns:\n",raw_data.columns)

print(raw_data.info())

Shape of the dataset:
 (11000, 17)

First few rows:
    age         job  marital  education default  balance housing loan  contact  \
0   59      admin.  married  secondary      no     2343     yes   no  unknown   
1   56      admin.  married  secondary      no       45      no   no  unknown   
2   41  technician  married  secondary      no     1270     yes   no  unknown   
3   55    services  married  secondary      no     2476     yes   no  unknown   
4   54      admin.  married   tertiary      no      184      no   no  unknown   

   day month  duration  campaign  pdays  previous poutcome deposit  
0    5   may      1042         1     -1         0  unknown     yes  
1    5   may      1467         1     -1         0  unknown     yes  
2    5   may      1389         1     -1         0  unknown     yes  
3    5   may       579         1     -1         0  unknown     yes  
4    5   may       673         2     -1         0  unknown     yes  

List of columns:
 Index(['age', 'job', 'marit

In [50]:
#Cardinality of variables
unique_counts = raw_data.nunique()
print(unique_counts)


age            76
job            12
marital         3
education       4
default         2
balance      3783
housing         2
loan            2
contact         3
day            31
month          12
duration     1423
campaign       36
pdays         472
previous       34
poutcome        4
deposit         2
dtype: int64


In [51]:
# Encoding object-type variables
df_encoded = pd.get_dummies(raw_data, columns=['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'poutcome', 'deposit'])

#Converting all boolean columns to integers (1 for True, 0 for False)
df_encoded[df_encoded.select_dtypes(include=['bool']).columns] = df_encoded.select_dtypes(include=['bool']).astype(int)
print(df_encoded.columns)
print(df_encoded.head())



Index(['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous',
       'job_admin.', 'job_blue-collar', 'job_entrepreneur', 'job_housemaid',
       'job_management', 'job_retired', 'job_self-employed', 'job_services',
       'job_student', 'job_technician', 'job_unemployed', 'job_unknown',
       'marital_divorced', 'marital_married', 'marital_single',
       'education_primary', 'education_secondary', 'education_tertiary',
       'education_unknown', 'default_no', 'default_yes', 'housing_no',
       'housing_yes', 'loan_no', 'loan_yes', 'contact_cellular',
       'contact_telephone', 'contact_unknown', 'month_apr', 'month_aug',
       'month_dec', 'month_feb', 'month_jan', 'month_jul', 'month_jun',
       'month_mar', 'month_may', 'month_nov', 'month_oct', 'month_sep',
       'poutcome_failure', 'poutcome_other', 'poutcome_success',
       'poutcome_unknown', 'deposit_no', 'deposit_yes'],
      dtype='object')
   age  balance  day  duration  campaign  pdays  previous  job

In [52]:
# Filter rows where pdays == -1
count_minus1 = (raw_data['pdays'] == -1).sum()

print(count_minus1)

# We see that there are 8203 where pdays = -1, that is where we have no info or contact. 


8203


In [53]:
# We only calculate the median on values greater than -1
known_pdays = df_encoded[df_encoded['pdays'] > -1]['pdays']
median_pdays = known_pdays.median()

# We replace all -1 values with the median of the know_pdays
df_encoded['Pdays_Duration'] = df_encoded['pdays'].replace(-1, median_pdays)

# We drop the original pdays variable
df_encoded.drop('pdays', axis=1, inplace=True)


In [55]:
print(df_encoded['Pdays_Duration'].tail())
print(df_encoded['Pdays_Duration'].iloc[1000:1020])

11157    182
11158    182
11159    182
11160    172
11161    182
Name: Pdays_Duration, dtype: int64
1009    182
1010    182
1011    182
1012    167
1013    182
1014    182
1015    182
1016    122
1017    182
1018    182
1019    182
1020    182
1021    182
1022    182
1023    182
1024    183
1025    195
1026    133
1027    182
1028    182
Name: Pdays_Duration, dtype: int64
